In [2]:
from functools import partial
from collections import namedtuple
import os
from pytorch_toolbelt.utils import read_rgb_image
from predictor import FaceMeshPredictor
import cv2
import os
import mediapipe as mp
from model_training.utils import load_indices_from_npy
from utils import get_relative_path
import numpy as np

In [3]:
def run_dad_net_custom_points(image_path, indices_file):

    # pass a numpy indices to get those points 
    
    image = read_rgb_image(image_path)
    predictor = FaceMeshPredictor.dad_3dnet()
    predictions = predictor(image)
    projected_vertices = predictions["projected_vertices"].squeeze().numpy().astype(int) # projected vertices (total of 5023)
    #print(projected_vertices.shape)
    indices = np.load(indices_file)
    points = []
    points.extend(np.take(projected_vertices, indices, axis=0))

    points = np.array(points)

    return points

In [4]:
def run_dad_net_dir(image_dir, output_dir, indices_file):
    images = os.listdir(image_dir)

    for image in images:
        filename = image.split('.')[0]
        curr_image = os.path.join(image_dir, image)
        output_path = os.path.join(output_dir, filename+'.npy')
        curr_points = run_dad_net_custom_points(curr_image, indices_file)
        np.save(output_path, curr_points)
    
    return

In [6]:
image_dir = '../my_code/2D_lmk_ideas/dense_2d/sample_data/images/'
output_dir = '../my_code/2D_lmk_ideas/dense_2d/sample_data/dadnet_2d/'
indices_file = "./model_training/model/static/face_keypoints/ids1.npy"

run_dad_net_dir(image_dir, output_dir, indices_file)

/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return forward_call(*input, **kwargs)
